In [1]:
# some imports we will need in this example

import multiDGD
import numpy as np
import anndata as ad
import scanpy as sc
import pandas as pd

In [ ]:
save_dir = "../tutorials/"

### here I created an anndata object that has the train-val-test split I used

In [7]:
"""
# load the data and save it in the new format
data = ad.read_h5ad(
    save_dir
    + "data/human_bonemarrow/GSE194122_openproblems_neurips2021_multiome_BMMC_processed.h5ad"
)

# load the train-val-test split
is_train_df = pd.read_csv(save_dir + "data/human_bonemarrow/train_val_test_split.csv")
# the old one used other terms so we need to replace them
# replace all 'test' with 'validation'
is_train_df["is_train"] = is_train_df["is_train"].replace("test", "validation")
# replace all 'iid_holdout' with 'test'
is_train_df["is_train"] = is_train_df["is_train"].replace("iid_holdout", "test")
# add this to the data object
data.obs["train_val_test"] = is_train_df["is_train"].values

# prepare data set for model
data = multiDGD.functions._data.setup_data(
    data,
    modality_key="feature_types",
    observable_key="cell_type",
    covariate_keys=["Site"],
)
# save this data to keep the train-val-test split for later
data.write_h5ad(
    save_dir
    + "data/human_bonemarrow/GSE194122_openproblems_neurips2021_multiome_BMMC_processed_withDataSplit.h5ad"
)
"""

In [2]:
# load data and export the train_val_test to a csv file
data = ad.read_h5ad(
    save_dir
    + "data/human_bonemarrow/GSE194122_openproblems_neurips2021_multiome_BMMC_processed_withDataSplit.h5ad"
)

In [23]:
model = multiDGD.DGD(
    data=data,
    save_dir=save_dir + "models/",
    model_name="dgd_bonemarrow_default_trained_and_tested",
)

selected  22  number of Gaussian mixture components based on the provided observable.
Covariate model initialized as:

        Gaussian_mix_compture:
            Dimensionality: 2
            Number of components: 4
        


### access the trained model's parameters and save for this version of the model

In [31]:
# name of the model files in ../tutorials/models/
dgd_name = "human_bonemarrow_l20_h2-3_test50e"

# load the parameter dictionary as an ordered dictionary
# import collections
import torch

checkpoint = torch.load(
    "../tutorials/models/" + dgd_name + ".pt", map_location=torch.device("cpu")
)

In [26]:
# go through every entry of the OrderedDict and assign the values to the corresponding model parameters
for key, value in checkpoint.items():
    # if a key is not in the model parameters, we add it
    if key not in model.state_dict():
        model.state_dict()[key] = value
    else:
        model.state_dict()[key].copy_(value)

In [30]:
# check if it worked

checkpoint["gmm.mean"] == model.state_dict()["gmm.mean"]

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True],
        [True, True, True, 

In [32]:
# save this model
model.save()

In [33]:
# see if loading works

model = multiDGD.DGD.load(
    data=data,
    save_dir="../tutorials/models/",
    model_name="dgd_bonemarrow_default_trained_and_tested",
)

Covariate model initialized as:

        Gaussian_mix_compture:
            Dimensionality: 2
            Number of components: 4
        
#######################
Training status
#######################
True
